<a href="https://colab.research.google.com/github/Aditikumari20/NLP/blob/main/Amazon_Food_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sagemaker

In [4]:
import sagemaker
import boto3
import pandas as pd

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
prefix = "nlp-model-demo"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:5                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:265 in __init__                     │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:293 in _initialize                  │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [6]:
# Load dataset
df = pd.read_csv("Reviews.csv")  # Ensure the dataset is available locally
# Keep only required columns
df = df[["Text", "Score"]].dropna()
# Convert scores into binary sentiment (1 = Positive, 0 = Negative)
df["Sentiment"] = df["Score"]apply(lambda x: 1 if x > 3 else 0)
df = df[["Text", "Sentiment"]]
# Save processed data
df.to_csv("processed_reviews.csv", index=False)
# Upload to S3
s3 = boto3.client("s3")
s3.upload_file("processed_reviews.csv", bucket, f"{prefix}/processed_reviews.csv")
# Store S3 path
s3_train_data = f"s3://{bucket}/{prefix}/processed_reviews.csv"
print("Data uploaded to:", s3_train_data)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ df["Sentiment"] = df["Score"]apply(lambda x: 1 if x > 3 else 0)                                  │
│                              ▲                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [7]:
# Load dataset
df = pd.read_csv("Reviews.csv")  # Ensure the dataset is available locally
# Keep only required columns
df = df[["Text", "Score"]].dropna()
# Convert scores into binary sentiment (1 = Positive, 0 = Negative)
df["Sentiment"] = df["Score"]apply(lambda x: 1 if x > 3 else 0)
df = df[["Text", "Sentiment"]]
# Save processed data
df.to_csv("processed_reviews.csv", index=False)
# Upload to S3
s3 = boto3.client("s3")
s3.upload_file("processed_reviews.csv", bucket, f"{prefix}/processed_reviews.csv")
# Store S3 path
s3_train_data = f"s3://{bucket}/{prefix}/processed_reviews.csv"
print("Data uploaded to:", s3_train_data)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ df["Sentiment"] = df["Score"]apply(lambda x: 1 if x > 3 else 0)                                  │
│                              ▲                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [8]:
%%writefile train.py
import argparse
import os
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def train():
    # Argument parser for SageMaker input
    parser = argparse.ArgumentParser()
    parser.add_argument("--train_data", type=str, default=os.environ["SM_CHANNEL_TRAIN"])
    args = parser.parse_args()
    # Load dataset from the provided path
    train_data_path = os.path.join(args.train_data, "processed_reviews.csv")
    df = pd.read_csv(train_data_path)
    # Split data
    X = df["Text"]
    y = df["Sentiment"]
    # Create a text-processing pipeline
    pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(stop_words="english")),
        ("clf", LogisticRegression())
    ])
    # Train model
    pipeline.fit(X, y)
    # Save trained model
    model_path = os.path.join("/opt/ml/model", "model.joblib")
    joblib.dump(pipeline, model_path)
    print("Model saved at", model_path)
if _name_ == "_main_":
    train()

Writing train.py


In [9]:
from sagemaker.sklearn.estimator import SKLeran

sklearn_estimator = SKLearn(
    entry_point="train.py",
    framework_version="0.23-1",
    instance_type="ml.m5.large",
    role=role,
    sagemaker_session=sagemaker_session,
)
sklearn_estimator.fit({"train": s3_train_data})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 0>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'SKLeran' from 'sagemaker.sklearn.estimator' 
(/usr/local/lib/python3.11/dist-packages/sagemaker/sklearn/estimator.py)

In [10]:
%%writefile inference.py
import joblib
import os
import json
import pandas as pd
# Load trained model
def model_fn(model_dir):
    model_path = os.path.join(model_dir, "model.joblib")
    return joblib.load(model_path)
# Parse input JSON
def input_fn(request_body, request_content_type):
    if request_content_type == "application/json":
        data = json.loads(request_body)
        return pd.DataFrame(data, columns=["Text"])
    else:
        raise ValueError("Unsupported content type: {}".format(request_content_type))
# Generate predictions
def predict_fn(input_data, model):
    return model.predict(input_data["Text"]).tolist()

Writing inference.py


Deploy The Model In SageMaker

In [11]:
from sagemaker.sklearn.modelimport SKLearnModel
model_data = sklearn_estimator.model_data

sklearn_model = SKLearnModel(
    model_data=model_data,
    role=role,
    entrypoint="inference.py",
    framework_version="0.23-1",
    sagemaker_session=sagemaker_session,
)
predictor = sklearn_model.deploy(instance_type="ml.m5.large",initial_instance_count=1)


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ from sagemaker.sklearn.modelimport SKLearnModel                                                  │
│                                    ▲                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [12]:
import json
test_data = json.dumps(["This product is amazing!", "Worst product ever."])
print("Predictions:", response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'response' is not defined

CleanUp Response

In [13]:
predictor.delete_endpoint()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictor' is not defined